1️⃣ Obter o deflator IPCA (pandas)

Para se evitar comparações enganosas de uma série temporal é importante deflacionar os valores com base em algum índice de preços. Com isso pode-se analisar o valor real ao longo do tempo neutralizando os efeitos da inflação. 

Isso permite comparações mais consistentes entre anos diferentes, e ao tomador de decisão ter embasamento mais preciso com base em dados reais.

O índice que será usado será o IPCA(Índice de Preços ao Consumidor Amplo) pois é o índice oficial de inflação usado pelo Banco Central do Brasil para definir a meta de inflação.

O IPEADATA é uma fonte confiável de dados macroeconômicos brasileiros, e usar o ipeadatapy facilita muito a coleta.

In [0]:
%pip install ipeadatapy

import ipeadatapy
import pandas as pd

# Série IPCA geral (mensal)
ipca_data = ipeadatapy.timeseries("PRECOS12_IPCA12")
ipca_data = ipca_data.reset_index()

print(ipca_data.head())

# Ajustar nomes de colunas
ipca_data.rename(columns={'VALUE (-)': 'value', 'DATE': 'date'}, inplace=True)

# Inputs
ano_inicial = 2013      
ano_final = 2024        
ano_base = 2024         # valores deflacionados a preços de 2024

# 1) Filtrar apenas meses de dezembro
ipca_dezembro = ipca_data[ipca_data['date'].dt.month == 12].copy()

# 2) Criar coluna 'Ano' e filtrar período desejado
ipca_dezembro['Ano'] = ipca_dezembro['date'].dt.year
ipca_anual = ipca_dezembro[
    (ipca_dezembro['Ano'] >= ano_inicial) &
    (ipca_dezembro['Ano'] <= ano_final)
]

# 3) Obter valor do IPCA do ano-base
valor_base_row = ipca_anual[ipca_anual['Ano'] == ano_base]
if not valor_base_row.empty:
    valor_base = valor_base_row['value'].iloc[0]
else:
    valor_base = ipca_anual['value'].iloc[-1]
    print(
        f"Aviso: IPCA do ano base {ano_base} não encontrado. "
        f"Usando o último ano disponível: {ipca_anual['Ano'].iloc[-1]}"
    )

# 4) Calcular deflator de cada ano
ipca_anual['deflator'] = valor_base / ipca_anual['value']

# DataFrame só com Ano + deflator
ipca_deflator = ipca_anual[['Ano', 'deflator']]
print(ipca_deflator)


2️⃣ Levar o deflator para Spark e unir com a SILVER

Agora será conectado isso com a tabela workspace.mvp.silver_dca:

In [0]:
from pyspark.sql import functions as F

# Converter o deflator (pandas) para Spark
df_deflator = spark.createDataFrame(ipca_deflator)

display(df_deflator)

# Carregar a tabela SILVER
df_silver = spark.table("workspace.mvp.silver_dca")

display(df_silver.limit(5))

# Juntar por Ano
df_silver_defl = df_silver.join(df_deflator, on="Ano", how="left")
display(df_silver_defl.limit(10))


3️⃣ Aplicar o deflator nas colunas econômica

Aqui é usado exatamente o vetor colunas_deflacionar:

In [0]:
# Colunas que serão deflacionadas
colunas_deflacionar = [
    "Receita_Total", "Receita_Exceto_Intra", "Receita_Corrente", "Receita_Capital",
    "Receita_Intra", "Receita_Intra_Corrente", "Receita_Intra_Capital",
    "Despesa_Total", "Despesa_Corrente", "Despesa_Capital",
    "Investimento"
]

# Criar novas colunas *_Real já deflacionadas
df_silver_defl_real = df_silver_defl

for col in colunas_deflacionar:
    df_silver_defl_real = df_silver_defl_real.withColumn(
        f"{col}_Real",
        F.col(col) * F.col("deflator")
    )

display(df_silver_defl_real.limit(10))
df_silver_defl_real.printSchema()


4️⃣ Salvar como tabela deflacionada (Silver “plus”)

In [0]:
df_silver_defl_real.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.mvp.silver_dca_deflacionada")

print("✅ Tabela criada: workspace.mvp.silver_dca_deflacionada")


5 Conferir a tabela deflacionada

In [0]:
df_silver_defl = spark.table("workspace.mvp.silver_dca_deflacionada")

display(df_silver_defl.limit(10))
df_silver_defl.printSchema()
